# 概要　

## スクレイピングの流れ
1. **各サイトのカレンダーページにアクセスし、開催日URLを取得する**
2. **取得した開催日URLにアクセスし、各開催日のレース情報にアクセスするためのレースIDを取得する**
3. **固定URLとレースIDを組み合わせて、各レースの詳細情報を取得する**

---

## 対象のサイト
- **netkeiba**
- **競馬ブック**

---

## 取り込み対象のデータ1

| 対象データ  | 取得サイト | 取得対象のソース        |
|--------------|------------|------------------------|
| 開催日URL     | netkeiba   | 開催日カレンダーページ  |
| レースID     | netkeiba   | 開催情報ページ          |
| 開催日URL     | 競馬ブック | 開催日カレンダーページ  |
| レースID     | 競馬ブック | 開催情報ページ          |

---

## 取り込み対象のデータ2

| Targetのインポート先 (カラム名)     | 取得サイト   | 取得対象のソース                        | インポートファイルの場所                       | 備考                                             |
|-------------------------------------|--------------|----------------------------------------|----------------------------------------------|------------------------------------------------|
| KOL関係者コメント                    | 競馬ブック    | レース結果ページ > 関係者インタビュー    | D:\Keiba\Importdata\10_Interview_Comments    |                                                |
| KOL次走へのメモ                      | 競馬ブック    | レース結果ページ > 次走へのメモ          | D:\Keiba\Importdata\10_Next_Comments         |                                                |
| レースコメント                      | netkeiba    | データベース > レース結果ページ > 馬場コメント              | D:\Keiba\Importdata\10_Race_Comments         |                            |
| 結果コメント                        | netkeiba      | データベース > レース結果ページ > 分析コメント | D:\Keiba\Importdata\10_Result_Comments       |                                                |
| 結果コメント                        | 競馬ブック    | レース結果ページ > 発送状況他            | D:\Keiba\Importdata\10_Result_Comments       | netkeibaの分析コメントに追加する形でインポート |
| 予想コメント                        | 競馬ブック      | 調教ページ                              | D:\Keiba\Importdata\10_Traning_Comments      |           |
| レース印1                           | netkeiba      | データベース > レース結果ページ > 馬場指数 | D:\Keiba\Importdata\20_Course_Condition      |                                                |
| 馬印3                               | 競馬ブック    | レース結果ページ > 各コーナー通過順      | D:\Keiba\Importdata\30_Corners_Position      |                                                |
| 馬印4                               | 競馬ブック    | レース結果ページ > 各コーナー通過順      | D:\Keiba\Importdata\30_Corners_Position      |                                                |
| 馬印5                               | 競馬ブック    | レース結果ページ > 各コーナー通過順      | D:\Keiba\Importdata\30_Corners_Position      |                                                |
| 馬印6                               | 競馬ブック    | レース結果ページ > 各コーナー通過順      | D:\Keiba\Importdata\30_Corners_Position      |                                                |
| 馬印7                               | 競馬ブック    | レース結果ページ > 4角位置              | D:\Keiba\Importdata\30_Spurt_Position        |                                                |
| 外部指数1                           | 競馬ブック    | 出馬表ページ > レイティング              | D:\Keiba\Importdata\40_Rathing_Score         |                                                |



# 回顧データ取得１
## ・netkeibaから指定した期間のレース結果情報を取得する
## ・TargetにインポートするためのCSVファイルを生成する

### スクレイピング対象データ
- 馬番
- 発送状況他
- レース分析コメント
- 馬場コメント
- 馬場指数

### ステップ
1. **netkeibaのカレンダーページから開催日を取得**
2. **取得した開催日と固定のURL要素を組み合わせ、開催情報ページにアクセスし、レース詳細ページにアクセスするためのレースIDを取得**
3. **取得した開催日とレースIDを組み合わせ、Target取り込み用のレースIDを生成**
4. **取得したレースIDと固定のURL要素を組み合わせ、レース結果ページへアクセス**
5. **レース結果ページから馬番を取得し、Target取り込み用のレースIDと組み合わせて、馬番付きレースIDを生成**
6. **レース結果ページから各データを取得し、以下のリストを生成**
   - `[馬番付きレースID, 発送状況他 & レース分析コメント]`
   - `[レースID, 馬場コメント]`
   - `[レースID, 馬場指数]`
7. **生成したリストをCSVファイルとして指定のディレクトリに出力**

In [10]:
import race_data_pipeline1
import tkinter as tk
import pandas as pd
import datetime
import importlib
import unicodedata

from tkcalendar import Calendar
from pathlib import Path

importlib.reload(race_data_pipeline1)

# 出力先パス
# netkeiba:成績データ
output_filepath1 = Path(r"D:\Keiba\00_ImportData\00_ImportData_Master\nk_Race_Master.csv")
# 競馬ブック:成績データ
output_filepath2 = Path(r"D:\Keiba\00_ImportData\00_ImportData_Master\kb_Race_Master.csv")
# 競馬ブック:発走状況他データ
output_filepath3 = Path(r"D:\Keiba\00_ImportData\00_ImportData_Master\kb_Status_Master.csv")
# 競馬ブック:各コーナーデータ
output_filepath4 = Path(r"D:\Keiba\00_ImportData\00_ImportData_Master\kb_First_Corner_Master.csv")
output_filepath5 = Path(r"D:\Keiba\00_ImportData\00_ImportData_Master\kb_Second_Corner_Master.csv")
output_filepath6 = Path(r"D:\Keiba\00_ImportData\00_ImportData_Master\kb_Third_Corner_Master.csv")
output_filepath7 = Path(r"D:\Keiba\00_ImportData\00_ImportData_Master\kb_Fourth_Corner_Master.csv")
# 競馬ブック:インタビューデータ
output_filepath8 = Path(r"D:\Keiba\00_ImportData\00_ImportData_Master\kb_Interview_Master.csv")
# 競馬ブック:次走へのメモデータ
output_filepath9 = Path(r"D:\Keiba\00_ImportData\00_ImportData_Master\kb_Next_Comments_Master.csv")

# レースコメント
output_filepath10 = Path(r"D:\Keiba\00_ImportData\10_Race_Comments\Race_Comments_Master.csv")
# 結果コメント
output_filepath11 = Path(r"D:\Keiba\00_ImportData\20_After_Comments\After_Comments_Master.csv")
# インタビューコメント
output_filepath12 = Path(r"D:\Keiba\00_ImportData\30_Interview_Comments\Interview_Comments_Master.csv")
# 次走へのメモ
output_filepath13 = Path(r"D:\Keiba\00_ImportData\40_Next_Comments\Next_Comments_Master.csv")
# 初角位置
output_filepath14 = Path(r"D:\Keiba\00_ImportData\70_First_Corner_Position\First_Corner_Position_Master.csv")
# 2角位置
output_filepath15 = Path(r"D:\Keiba\00_ImportData\80_Second_Corner_Position\Second_Corner_Position_Master.csv")
# 3角位置
output_filepath16 = Path(r"D:\Keiba\00_ImportData\90_Third_Corner_Position\Third_Corner_Position_Master.csv")
# 4角位置
output_filepath17 = Path(r"D:\Keiba\00_ImportData\100_Fourth_Corner_Position\Fourth_Corner_Position_Master.csv")
# 上り位置
output_filepath18 = Path(r"D:\Keiba\00_ImportData\110_Spurt_Position\Spurt_Position_Master.csv")
# 馬場指数
output_filepath19 = Path(r"D:\Keiba\00_ImportData\60_Track_Condition\Track_Condition_Master.csv")
# 前半3Fタイム
output_filepath20 = Path(r"D:\Keiba\00_ImportData\120_First3F_Lap\First3F_Lap_Master.csv")

def select_year_month(prompt_title: str) -> str:
    """
    カレンダーを表示して日付を一つ選ばせ、
    YYYY-MM-DD形式の文字列をパースして datetime.date にし、
    そこから "YYYY-MM" を返す。
    """
    today = datetime.date.today()

    root = tk.Tk()
    root.title(prompt_title)

    cal = Calendar(
        root,
        selectmode='day',
        year=today.year,
        month=today.month,
        day=today.day,
        locale='ja_JP',
        date_pattern='yyyy-mm-dd'
    )
    cal.pack(padx=20, pady=20)

    selected = {}

    def on_select():
        selected['date_str'] = cal.get_date()
        root.destroy()

    tk.Button(root, text='OK', command=on_select).pack(pady=10)
    root.mainloop()

    date_obj = datetime.datetime.strptime(selected['date_str'], '%Y-%m-%d').date()
    return date_obj.strftime('%Y-%m')

from_ym = select_year_month('Select start year-month')
to_ym = select_year_month('Select end year-month')

# スクレイピングする期間を表示
print(f"Scraping data from {from_ym} to {to_ym}...")

# netkeibaと競馬ブックの開催日とレースIDを取得
nk_kaisai_date_list = race_data_pipeline1.scrape_nk_kaisai_date(from_=from_ym, to_=to_ym)
nk_raceid_list, nk_target_raceid_list = race_data_pipeline1.scrape_nk_raceid_list(nk_kaisai_date_list)

kb_kaisai_date_list = race_data_pipeline1.scrape_kb_kaisai_date(from_=from_ym, to_=to_ym)
kb_raceid_list, kb_target_raceid_list = race_data_pipeline1.scrape_kb_raceid_list(kb_kaisai_date_list)

# netkeibaと競馬ブックのレースデータを取得
nk_race_df = race_data_pipeline1.scrape_nk_racedata(nk_raceid_list, nk_target_raceid_list)

(
    result_df,
    kb_status_df,
    first_corner_df,
    second_corner_df,
    third_corner_df,
    fourth_corner_df,
    interview_df,
    next_comments_df,
) = race_data_pipeline1.scrape_kb_racedata(
    kb_raceid_list,
    kb_target_raceid_list,
)

nk_temp = nk_race_df.copy()
result_temp = result_df.copy()
status_temp = kb_status_df.copy()
first_corner_temp = first_corner_df.copy()
second_corner_temp = second_corner_df.copy()
third_corner_temp = third_corner_df.copy()
fourth_corner_temp = fourth_corner_df.copy()
interview_temp = interview_df.copy()
next_comments_temp = next_comments_df.copy()

# 各データフレームへyear列追加
# nktemp:'target_horseid' の先頭4文字を年として抽出
nk_temp['year'] = pd.to_datetime(
    nk_temp['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

# result_temp:'target_horseid' の先頭4文字を年として抽出
result_temp['year'] = pd.to_datetime(
    result_temp['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

# status_temp:'target_horseid' の先頭4文字を年として抽出
status_temp['year'] = pd.to_datetime(
    status_temp['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

# first_corner_temp:'target_horseid' の先頭4文字を年として抽出
first_corner_temp['year'] = pd.to_datetime(
    first_corner_temp['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

# second_corner_temp:'target_horseid' の先頭4文字を年として抽出
second_corner_temp['year'] = pd.to_datetime(
    second_corner_temp['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

# third_corner_temp:'target_horseid' の先頭4文字を年として抽出
third_corner_temp['year'] = pd.to_datetime(
    third_corner_temp['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

# fourth_corner_temp:'target_horseid' の先頭4文字を年として抽出
fourth_corner_temp['year'] = pd.to_datetime(
    fourth_corner_temp['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

# interview_temp:'target_raceid' の先頭4文字を年として抽出
interview_temp['year'] = pd.to_datetime(
    interview_temp['target_raceid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

# next_comments_temp:'target_raceid' の先頭4文字を年として抽出
next_comments_temp['year'] = pd.to_datetime(
    next_comments_temp['target_raceid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

# インタビュー・次走へのメモの結合処理

# 馬名の正規化（全半角・空白・括弧・長音など）
def normalize_name(s: pd.Series) -> pd.Series:
    s = s.astype(str).fillna('').map(lambda x: unicodedata.normalize('NFKC', x))
    s = s.str.replace(r'\s+', '', regex=True)       # 全空白除去
    s = s.str.replace(r'[・･·]', '', regex=True)    # 中黒
    s = s.str.replace(r'[（）()]', '', regex=True)   # 括弧
    s = s.str.replace(r'[－―ー]', '-', regex=True)  # 長音/ダッシュ
    return s

# result_tmp に race_id を作成（末尾2文字を削る）
result_tmp = result_temp.copy()
result_tmp['target_raceid'] = result_tmp['target_horseid'].astype(str).str.slice(0, -2)

# result_tmp 側のマージ用キー（race_norm, name_norm → horseid）
r_key = result_tmp.assign(
    race_norm = result_tmp['target_raceid'].astype(str),
    name_norm = normalize_name(result_tmp['馬名'])
)[['race_norm', 'name_norm', 'target_horseid']].drop_duplicates()

# interview_tempとの結合
i_key = interview_temp.assign(
    race_norm = interview_temp['target_raceid'].astype(str),
    name_norm = normalize_name(interview_temp['馬名'])
)[['race_norm', 'name_norm', 'KOL関係者コメント','year']]

merged_interview = i_key.merge(
    r_key, on=['race_norm','name_norm'], how='inner', validate='many_to_one'
)

# next_comments_tempとの結合
n_key = next_comments_temp.assign(
    race_norm = next_comments_temp['target_raceid'].astype(str),
    name_norm = normalize_name(next_comments_temp['馬名'])
)[['race_norm', 'name_norm', 'KOL次走へのメモ','year']]

merged_next = n_key.merge(
    r_key, on=['race_norm','name_norm'], how='inner', validate='many_to_one'
)

# 結果コメントの結合処理
# nk_tmp と status_tmp を target_horseid で結合
after_comments_df = pd.merge(
    nk_temp[['target_horseid', '結果コメント','year']], 
    status_temp[['target_horseid', '発走状況他']], 
    on='target_horseid', 
    how='left'
)

# 条件付きでカンマを入れる
after_comments_df['結果コメント'] = after_comments_df.apply(
    lambda x: f"{x['発走状況他']}/{x['結果コメント']}" if pd.notna(x['発走状況他']) else x['結果コメント'],
    axis=1
)

# レースコメントの抽出
add_df1 = nk_temp[['target_raceid', 'レースコメント','year']]

# 結果コメントの抽出
add_df2 = after_comments_df[['target_horseid', '結果コメント','year']]

# 関係者コメント抽出
add_df3 = merged_interview[['target_horseid','KOL関係者コメント','year']]

# 次走へのメモ抽出
add_df4 = merged_next[['target_horseid', 'KOL次走へのメモ','year']]

# 初角位置の抽出
add_df5 = first_corner_temp[['target_horseid', '初角サイドポジション','year']]

# 2角位置の抽出
add_df6 = second_corner_temp[['target_horseid', '2角サイドポジション','year']]

# 3角位置の抽出
add_df7 = third_corner_temp[['target_horseid', '3角サイドポジション','year']]

# 4角位置の抽出
add_df8 = fourth_corner_temp[['target_horseid', '4角サイドポジション','year']]

# 上り位置の抽出
add_df9 = result_tmp[['target_horseid', '4角位置','year']]

# 馬場指数の抽出
add_df10 = nk_temp[['target_raceid', '馬場指数', 'year']]

# 前半3Fタイムの抽出
add_df11 = result_tmp[['target_horseid', '前半3F', 'year']]

# マスタcsvファイルを開く
master_df1 = pd.read_csv(output_filepath1, encoding= 'cp932').copy()
master_df2 = pd.read_csv(output_filepath2, encoding= 'cp932').copy()
master_df3 = pd.read_csv(output_filepath3, encoding= 'cp932').copy()
master_df4 = pd.read_csv(output_filepath4, encoding= 'cp932').copy()
master_df5 = pd.read_csv(output_filepath5, encoding= 'cp932').copy()
master_df6 = pd.read_csv(output_filepath6, encoding= 'cp932').copy()
master_df7 = pd.read_csv(output_filepath7, encoding= 'cp932').copy()
master_df8 = pd.read_csv(output_filepath8, encoding= 'cp932').copy()
master_df9 = pd.read_csv(output_filepath9, encoding= 'cp932').copy()

# 派生マスタcsvファイルを開く
master_df10 = pd.read_csv(output_filepath10, encoding= 'cp932').copy()
master_df11 = pd.read_csv(output_filepath11, encoding= 'cp932').copy()
master_df12 = pd.read_csv(output_filepath12, encoding= 'cp932').copy()
master_df13 = pd.read_csv(output_filepath13, encoding= 'cp932').copy()
master_df14 = pd.read_csv(output_filepath14, encoding= 'cp932').copy()
master_df15 = pd.read_csv(output_filepath15, encoding= 'cp932').copy()
master_df16 = pd.read_csv(output_filepath16, encoding= 'cp932').copy()
master_df17 = pd.read_csv(output_filepath17, encoding= 'cp932').copy()
master_df18 = pd.read_csv(output_filepath18, encoding= 'cp932').copy()
master_df19 = pd.read_csv(output_filepath19, encoding= 'cp932').copy()
master_df20 = pd.read_csv(output_filepath20, encoding= 'cp932').copy()

# 新しいデータを追加
saved_df1 = pd.concat([master_df1, nk_temp], ignore_index=True)
saved_df2 = pd.concat([master_df2, result_temp], ignore_index=True)
saved_df3 = pd.concat([master_df3, status_temp], ignore_index=True)
saved_df4 = pd.concat([master_df4, first_corner_temp], ignore_index=True)
saved_df5 = pd.concat([master_df5, second_corner_temp], ignore_index=True)
saved_df6 = pd.concat([master_df6, third_corner_temp], ignore_index=True)
saved_df7 = pd.concat([master_df7, fourth_corner_temp], ignore_index=True)
saved_df8 = pd.concat([master_df8, interview_temp], ignore_index=True)
saved_df9 = pd.concat([master_df9, next_comments_temp], ignore_index=True)
saved_df10 = pd.concat([master_df10, add_df1], ignore_index=True)
saved_df11 = pd.concat([master_df11, add_df2], ignore_index=True)
saved_df12 = pd.concat([master_df12, add_df3], ignore_index=True)
saved_df13 = pd.concat([master_df13, add_df4], ignore_index=True)
saved_df14 = pd.concat([master_df14, add_df5], ignore_index=True)
saved_df15 = pd.concat([master_df15, add_df6], ignore_index=True)
saved_df16 = pd.concat([master_df16, add_df7], ignore_index=True)
saved_df17 = pd.concat([master_df17, add_df8], ignore_index=True)
saved_df18 = pd.concat([master_df18, add_df9], ignore_index=True)
saved_df19 = pd.concat([master_df19, add_df10], ignore_index=True)
saved_df20 = pd.concat([master_df20, add_df11], ignore_index=True)

# キーが重複している場合、最新のデータ（最後の行）を残して削除
saved_df1 = saved_df1.drop_duplicates(subset=['target_horseid'], keep='last')
saved_df2 = saved_df2.drop_duplicates(subset=['target_horseid'], keep='last')
saved_df3 = saved_df3.drop_duplicates(subset=['target_horseid'], keep='last')
saved_df4 = saved_df4.drop_duplicates(subset=['target_horseid'], keep='last')
saved_df5 = saved_df5.drop_duplicates(subset=['target_horseid'], keep='last')
saved_df6 = saved_df6.drop_duplicates(subset=['target_horseid'], keep='last')
saved_df7 = saved_df7.drop_duplicates(subset=['target_horseid'], keep='last')
saved_df8 = saved_df8.drop_duplicates(subset=['target_raceid'], keep='last')
saved_df9 = saved_df9.drop_duplicates(subset=['target_raceid'], keep='last')
saved_df10 = saved_df10.drop_duplicates(subset=['target_raceid'], keep='last')
saved_df11 = saved_df11.drop_duplicates(subset=['target_horseid'], keep='last')
saved_df12 = saved_df12.drop_duplicates(subset=['target_horseid'], keep='last')
saved_df13 = saved_df13.drop_duplicates(subset=['target_horseid'], keep='last')
saved_df14 = saved_df14.drop_duplicates(subset=['target_horseid'], keep='last')
saved_df15 = saved_df15.drop_duplicates(subset=['target_horseid'], keep='last')
saved_df16 = saved_df16.drop_duplicates(subset=['target_horseid'], keep='last')
saved_df17 = saved_df17.drop_duplicates(subset=['target_horseid'], keep='last')
saved_df18 = saved_df18.drop_duplicates(subset=['target_horseid'], keep='last')
saved_df19 = saved_df19.drop_duplicates(subset=['target_raceid'], keep='last')
saved_df20 = saved_df20.drop_duplicates(subset=['target_horseid'], keep='last')

# 対象のキーを持つデータフレームをまとめて型統一
for df, keycol in [
    (saved_df1, 'target_horseid'),
    (saved_df2, 'target_horseid'),
    (saved_df3, 'target_horseid'),
    (saved_df4, 'target_horseid'),
    (saved_df5, 'target_horseid'),
    (saved_df6, 'target_horseid'),
    (saved_df7, 'target_horseid'),
    (saved_df8, 'target_raceid'),
    (saved_df9, 'target_raceid'),
    (saved_df10, 'target_raceid'),
    (saved_df11, 'target_horseid'),
    (saved_df12, 'target_horseid'),
    (saved_df13, 'target_horseid'),
    (saved_df14, 'target_horseid'),
    (saved_df15, 'target_horseid'),
    (saved_df16, 'target_horseid'),
    (saved_df17, 'target_horseid'),
    (saved_df18, 'target_horseid'),
    (saved_df19, 'target_raceid'),
    (saved_df20, 'target_horseid'),
]:
    df[keycol] = df[keycol].astype(str)
    df.sort_values(by=keycol, inplace=True, kind='mergesort', na_position='last')

# 保存前にキーでソート
saved_df1 = saved_df1.sort_values(by=['target_horseid'])
saved_df2 = saved_df2.sort_values(by=['target_horseid'])
saved_df3 = saved_df3.sort_values(by=['target_horseid'])
saved_df4 = saved_df4.sort_values(by=['target_horseid'])
saved_df5 = saved_df5.sort_values(by=['target_horseid'])
saved_df6 = saved_df6.sort_values(by=['target_horseid'])
saved_df7 = saved_df7.sort_values(by=['target_horseid'])
saved_df8 = saved_df8.sort_values(by=['target_raceid'])
saved_df9 = saved_df9.sort_values(by=['target_raceid'])
saved_df10 = saved_df10.sort_values(by=['target_raceid'])
saved_df11 = saved_df11.sort_values(by=['target_horseid'])
saved_df12 = saved_df12.sort_values(by=['target_horseid'])
saved_df13 = saved_df13.sort_values(by=['target_horseid'])
saved_df14 = saved_df14.sort_values(by=['target_horseid'])
saved_df15 = saved_df15.sort_values(by=['target_horseid'])
saved_df16 = saved_df16.sort_values(by=['target_horseid'])
saved_df17 = saved_df17.sort_values(by=['target_horseid'])
saved_df18 = saved_df18.sort_values(by=['target_horseid'])
saved_df19 = saved_df19.sort_values(by=['target_raceid'])
saved_df20 = saved_df20.sort_values(by=['target_horseid'])

# マスタcsvファイルを保存
saved_df1.to_csv(output_filepath1, index=False, encoding='cp932')
saved_df2.to_csv(output_filepath2, index=False, encoding='cp932')
saved_df3.to_csv(output_filepath3, index=False, encoding='cp932')
saved_df4.to_csv(output_filepath4, index=False, encoding='cp932')
saved_df5.to_csv(output_filepath5, index=False, encoding='cp932')
saved_df6.to_csv(output_filepath6, index=False, encoding='cp932')
saved_df7.to_csv(output_filepath7, index=False, encoding='cp932')
saved_df8.to_csv(output_filepath8, index=False, encoding='cp932')
saved_df9.to_csv(output_filepath9, index=False, encoding='cp932')

# 派生マスタcsvファイルを保存
saved_df10.to_csv(output_filepath10, index=False, encoding='cp932')
saved_df11.to_csv(output_filepath11, index=False, encoding='cp932')
saved_df12.to_csv(output_filepath12, index=False, encoding='cp932')
saved_df13.to_csv(output_filepath13, index=False, encoding='cp932')
saved_df14.to_csv(output_filepath14, index=False, encoding='cp932')
saved_df15.to_csv(output_filepath15, index=False, encoding='cp932')
saved_df16.to_csv(output_filepath16, index=False, encoding='cp932')
saved_df17.to_csv(output_filepath17, index=False, encoding='cp932')
saved_df18.to_csv(output_filepath18, index=False, encoding='cp932')
saved_df19.to_csv(output_filepath19, index=False, encoding='cp932')
saved_df20.to_csv(output_filepath20, index=False, encoding='cp932')

# 派生マスタファイルを年度別にファイルを分けて保存（year列を削除）
for year, df in saved_df10.groupby('year'):
    df.drop(columns='year').to_csv(
        f"D:/Keiba/00_ImportData/10_Race_Comments/Race_Comments_Master_{year}.csv",
        index=False, encoding='cp932'
    )

for year, df in saved_df11.groupby('year'):
    df.drop(columns='year').to_csv(
        f"D:/Keiba/00_ImportData/20_After_Comments/After_Comments_Master_{year}.csv",
        index=False, encoding='cp932'
    )

for year, df in saved_df12.groupby('year'):
    df.drop(columns='year').to_csv(
        f"D:/Keiba/00_ImportData/30_Interview_Comments/Interview_Comments_Master_{year}.csv",
        index=False, encoding='cp932'
    )

for year, df in saved_df13.groupby('year'):
    df.drop(columns='year').to_csv(
        f"D:/Keiba/00_ImportData/40_Next_Comments/Next_Comments_Master_{year}.csv",
        index=False, encoding='cp932'
    )

for year, df in saved_df14.groupby('year'):
    df.drop(columns='year').to_csv(
        f"D:/Keiba/00_ImportData/70_First_Corner_Position/First_Corner_Position_Master_{year}.csv",
        index=False, encoding='cp932'
    )

for year, df in saved_df15.groupby('year'):
    df.drop(columns='year').to_csv(
        f"D:/Keiba/00_ImportData/80_Second_Corner_Position/Second_Corner_Position_Master_{year}.csv",
        index=False, encoding='cp932'
    )

for year, df in saved_df16.groupby('year'):
    df.drop(columns='year').to_csv(
        f"D:/Keiba/00_ImportData/90_Third_Corner_Position/Third_Corner_Position_Master_{year}.csv",
        index=False, encoding='cp932'
    )

for year, df in saved_df17.groupby('year'):
    df.drop(columns='year').to_csv(
        f"D:/Keiba/00_ImportData/100_Fourth_Corner_Position/Fourth_Corner_Position_Master_{year}.csv",
        index=False, encoding='cp932'
    )

for year, df in saved_df18.groupby('year'):
    df.drop(columns='year').to_csv(
        f"D:/Keiba/00_ImportData/110_Spurt_Position/Spurt_Position_Master_{year}.csv",
        index=False, encoding='cp932'
    )

for year, df in saved_df19.groupby('year'):
    df.drop(columns='year').to_csv(
        f"D:/Keiba/00_ImportData/60_Track_Condition/Track_Condition_Master_{year}.csv",
        index=False, encoding='cp932'
    )

for year, df in saved_df20.groupby('year'):
    df.drop(columns='year').to_csv(
        f"D:/Keiba/00_ImportData/120_First3F_Lap/First3F_Lap_Master_{year}.csv",
        index=False, encoding='cp932'
    )



Scraping data from 2023-07 to 2026-01...


  0%|          | 0/8916 [00:00<?, ?it/s]

[WARN] race_id=202302060101 missing columns=['前半3F'], columns=['着順', 'My印', '本紙', '枠番', '馬番', '馬名', '性齢', '重量', '騎手', '騎手.1', 'タイム', '着差', '通過順位', '4角位置', '平均ハロン', '上り3F', '単人気', '単勝オッズ', '馬体重', '増減', '厩舎']


  0%|          | 3/8916 [00:05<4:01:54,  1.63s/it]

[WARN] race_id=202302060104 missing columns=['前半3F'], columns=['着順', 'My印', '本紙', '枠番', '馬番', '馬名', '性齢', '重量', '騎手', '騎手.1', 'タイム', '着差', '通過順位', '4角位置', '平均ハロン', '上り3F', '単人気', '単勝オッズ', '馬体重', '増減', '厩舎']


...

100%|█████████▉| 8908/8916 [4:35:46<00:19,  2.41s/it]

[WARN] race_id=202601030305 missing columns=['前半3F'], columns=['着順', 'My印', '本紙', '枠番', '馬番', '馬名', '性齢', '重量', '騎手', '騎手.1', 'タイム', '着差', '通過順位', '4角位置', '平均ハロン', '上り3F', '単人気', '単勝オッズ', '馬体重', '増減', '厩舎']


100%|█████████▉| 8912/8916 [4:35:55<00:09,  2.42s/it]

[WARN] race_id=202601030309 missing columns=['前半3F'], columns=['着順', 'My印', '本紙', '枠番', '馬番', '馬名', '性齢', '重量', '騎手', '騎手.1', 'タイム', '着差', '通過順位', '4角位置', '前半2F', '上り3F', '単人気', '単勝オッズ', '馬体重', '増減', '厩舎']


100%|██████████| 8916/8916 [4:36:06<00:00,  1.86s/it]
d:\Keiba\Python\scripts\race_data_pipeline1.py:731: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._data = (pd.concat([self._data, df_append], ignore_index=True, axis=0)
d:\Keiba\Python\scripts\race_data_pipeline1.py:731: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._data = (pd.concat([self._data, df_append], ignore_index=True, axis=0)
C:\Users\rotte\AppData\Local\Temp\ipykernel_10536\2282737865.py:281: DtypeWarning: Columns (5) hav

# 前日データ取得
## ・競馬ブックから指定した期間に出走している馬のレイティングを取得する
## ・競馬ブックから指定した期間に出走している馬の追切情報を取得する
## ・取得したデータを元にTargetにインポートするためのCSVファイルを生成する

### スクレイピング対象データ
- 馬番
- 各馬レイティング
- 各馬追切情報

### ステップ
1. **netkeibaのカレンダーページから開催日を取得**
2. **開催日と固定のURL要素を組み合わせ、開催情報ページにアクセスし、レースIDを取得**
3. **開催日とレースIDを組み合わせて、Target取り込み用のレースIDを生成**
4. **競馬ブックへアクセスし、指定した期間のレースIDを取得**
5. **出馬表ページから馬番を取得し、馬番付きレースIDを生成**
6. **出馬表ページから各馬のレイティングを取得し、以下の形でデータフレームを生成**
   - `[馬番付きレースID, 追切情報]`
7. **調教ページから馬番を取得し、馬番付きレースIDを生成**
8. **調教ページから各馬の追切情報を取得し、以下の形でデータフレームを生成**
9. **指定のディレクトリへ各データフレームをcsv保存**
10. **指定のディレクトリへ年度別の各データフレームをcsv保存**


In [7]:
import race_data_pipeline2
import tkinter as tk
import pandas as pd
import datetime
import importlib

from tkcalendar import Calendar
from pathlib import Path

importlib.reload(race_data_pipeline2)

# 出力先パス
output_dir1 = Path(r"D:\Keiba\00_ImportData\200_Rating_Score\Rating_Score_Master.csv")
output_dir2 = Path(r"D:\Keiba\00_ImportData\50_Training_Comments\Training_Comments_Master.csv")

def select_year_month(prompt_title: str) -> str:
    """
    カレンダーを表示して日付を一つ選ばせ、
    YYYY-MM-DD形式の文字列をパースして datetime.date にし、
    そこから "YYYY-MM" を返す。
    """
    today = datetime.date.today()

    root = tk.Tk()
    root.title(prompt_title)

    cal = Calendar(
        root,
        selectmode='day',
        year=today.year,
        month=today.month,
        day=today.day,
        locale='ja_JP',
        date_pattern='yyyy-mm-dd'
    )
    cal.pack(padx=20, pady=20)

    selected = {}

    def on_select():
        selected['date_str'] = cal.get_date()
        root.destroy()

    tk.Button(root, text='OK', command=on_select).pack(pady=10)
    root.mainloop()

    date_obj = datetime.datetime.strptime(selected['date_str'], '%Y-%m-%d').date()
    return date_obj.strftime('%Y-%m')

from_ym = select_year_month('Select start year-month')
to_ym = select_year_month('Select end year-month')

# スクレイピングする期間を表示
print(f"Scraping data from {from_ym} to {to_ym}...")

# netkeibaの開催日リストとレースIDからTarget取り込み用のIDを生成
# nk_kaisai_date_list = race_data_pipeline2.scrape_nk_kaisai_date(from_=from_ym, to_=to_ym)
# target_raceid_list = race_data_pipeline2.scrape_nk_raceid_list(nk_kaisai_date_list)

# 競馬ブックの開催日リストとレースID
kb_kaisai_date_list = race_data_pipeline2.scrape_kb_kaisai_date(from_=from_ym, to_=to_ym)
kb_raceid_list, kb_target_raceid_list = race_data_pipeline2.scrape_kb_raceid_list(kb_kaisai_date_list)

# 競馬ブックのレイティングを取得
rating_df = race_data_pipeline2.scrape_kb_rating(kb_raceid_list, kb_target_raceid_list)

# 競馬ブックの追切データを取得
training_df = race_data_pipeline2.scrape_kb_training(kb_raceid_list, kb_target_raceid_list)

# 'target_horseid'の先頭8文字を日付としてパースし、年を抽出して'year'列に追加
rating_df['datetime'] = pd.to_datetime(
    rating_df['target_horseid'].astype(str).str.slice(0, 8),
    format='%Y%m%d',
    errors='coerce'
)

training_df['datetime'] = pd.to_datetime(
    training_df['target_horseid'].astype(str).str.slice(0, 8),
    format='%Y%m%d',
    errors='coerce'
)

rating_df['year'] = rating_df['datetime'].dt.year
training_df['year'] = training_df['datetime'].dt.year

# マスタCSVファイルを開く
master_df1 = pd.read_csv(output_dir1, encoding='cp932', dtype={'target_horseid': str}).copy()
master_df2 = pd.read_csv(output_dir2, encoding='cp932', dtype={'target_horseid': str}).copy()

# 新しいデータをマスタに追加
merged_df1 = pd.concat([master_df1, rating_df], ignore_index=True)
merged_df2 = pd.concat([master_df2, training_df], ignore_index=True)

# target_horseidで重複している場合、最新のデータを残す
merged_df1 = merged_df1.drop_duplicates(subset=['target_horseid'], keep='last')
merged_df2 = merged_df2.drop_duplicates(subset=['target_horseid'], keep='last')

# 保存前に照準ソート
saved_df1 = merged_df1.sort_values('target_horseid', key=lambda s: s.astype(str))
saved_df2 = merged_df2.sort_values('target_horseid', key=lambda s: s.astype(str))

# マスタCSVファイルを保存
saved_df1.to_csv(output_dir1, index=False, encoding='cp932')
saved_df2.to_csv(output_dir2, index=False, encoding='cp932')

# マスタファイルを年度別にファイルを分けて保存（year列を削除）
for year, df in saved_df1.groupby('year'):
    df.drop(columns=['datetime','year']).to_csv(
        f"D:/Keiba/00_ImportData/200_Rating_Score/Rating_Score_Master_{year}.csv",
        index=False, encoding='cp932'
    )

for year, df in saved_df2.groupby('year'):
    df.drop(columns=['datetime','year']).to_csv(
        f"D:/Keiba/00_ImportData/50_Training_Comments/Training_Comments_Master_{year}.csv",
        index=False, encoding='cp932'
    )

Scraping data from 2023-07 to 2025-12...


 47%|████▋     | 4032/8640 [2:51:19<6:40:22,  5.21s/it]

Error scraping race ID 202403070612: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x525093
	0x5250d4
	0x31b490
	0x3658da
	0x365b7b
	0x3a7b42
	0x388204
	0x3a5341
	0x387f56
	0x3596c9
	0x35a484
	0x777e34
	0x7730c9
	0x790add
	0x53db38
	0x5458ad
	0x52d848
	0x52da12
	0x5175fa
	0x76785d49
	0x77e9d5db
	0x77e9d561

Traceback (most recent call last):
  File "d:\Keiba\Python\scripts\race_data_pipeline2.py", line 461, in scrape_kb_rating
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "syutuba")))
    ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rotte\AppData\Local\Programs\Python\Python313\Lib\site-packages\selenium\webdriver\support\wait.py", line 138, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x525093
	0x5250d4
	0x31b490
	0x3658da
	0x365b7b
	0x3a7b42
	0x388204
	0x3a5

 54%|█████▍    | 4652/8640 [3:17:45<5:50:46,  5.28s/it]

Error scraping race ID 202406000108: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x525093
	0x5250d4
	0x31b490
	0x3658da
	0x365b7b
	0x3a7b42
	0x388204
	0x3a5341
	0x387f56
	0x3596c9
	0x35a484
	0x777e34
	0x7730c9
	0x790add
	0x53db38
	0x5458ad
	0x52d848
	0x52da12
	0x5175fa
	0x76785d49
	0x77e9d5db
	0x77e9d561

Traceback (most recent call last):
  File "d:\Keiba\Python\scripts\race_data_pipeline2.py", line 461, in scrape_kb_rating
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "syutuba")))
    ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rotte\AppData\Local\Programs\Python\Python313\Lib\site-packages\selenium\webdriver\support\wait.py", line 138, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x525093
	0x5250d4
	0x31b490
	0x3658da
	0x365b7b
	0x3a7b42
	0x388204
	0x3a5

 76%|███████▋  | 6609/8640 [4:47:26<3:10:20,  5.62s/it]

Error scraping race ID 202502041109: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x525093
	0x5250d4
	0x31b490
	0x3658da
	0x365b7b
	0x3a7b42
	0x388204
	0x3a5341
	0x387f56
	0x3596c9
	0x35a484
	0x777e34
	0x7730c9
	0x790add
	0x53db38
	0x5458ad
	0x52d848
	0x52da12
	0x5175fa
	0x76785d49
	0x77e9d5db
	0x77e9d561

Traceback (most recent call last):
  File "d:\Keiba\Python\scripts\race_data_pipeline2.py", line 461, in scrape_kb_rating
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "syutuba")))
    ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rotte\AppData\Local\Programs\Python\Python313\Lib\site-packages\selenium\webdriver\support\wait.py", line 138, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x525093
	0x5250d4
	0x31b490
	0x3658da
	0x365b7b
	0x3a7b42
	0x388204
	0x3a5

100%|██████████| 8640/8640 [6:20:08<00:00,  2.64s/it]  


[WARN] cyokyo table not found: 202403070612
[WARN] training page not found (cyokyo table missing): 202403070612
[WARN] cyokyo table not found: 202406000108
[WARN] training page not found (cyokyo table missing): 202406000108
[WARN] cyokyo table not found: 202502041109
[WARN] training page not found (cyokyo table missing): 202502041109


# ↓アーカイブ

In [37]:
master_df1 = pd.read_csv(output_dir1, encoding='cp932', dtype={'target_horseid': str}).copy()
master_df2 = pd.read_csv(output_dir2, encoding='cp932', dtype={'target_horseid': str}).copy()

def add_dt_year_from_thid(df: pd.DataFrame, col='target_horseid') -> pd.DataFrame:
    out = df.copy()
    s = out[col].astype(str)
    # 先頭から連続する8桁を抽出（見つからなければ NaN）
    first8 = s.str.extract(r'(\d{8})', expand=False)
    out['datetime'] = pd.to_datetime(first8, format='%Y%m%d', errors='coerce')
    out['year'] = out['datetime'].dt.year.astype('Int64')  # 欠損は <NA>
    return out

# 新規データ
rating_df   = add_dt_year_from_thid(rating_df)
training_df = add_dt_year_from_thid(training_df)

# マージ
merged_df1 = pd.concat([master_df1, rating_df], ignore_index=True)
merged_df2 = pd.concat([master_df2, training_df], ignore_index=True)

# target_horseid で重複解消（最新優先：必要に応じて実行）
merged_df1 = merged_df1.drop_duplicates(subset=['target_horseid'], keep='last')

# マージ後にも一括で正規化（旧マスタ由来の欠損も埋める）
merged_df1 = add_dt_year_from_thid(merged_df1)
merged_df2 = add_dt_year_from_thid(merged_df2)

# 照準ソート
saved_df1 = merged_df1.sort_values('target_horseid', key=lambda s: s.astype(str))
saved_df2 = merged_df2.sort_values('target_horseid', key=lambda s: s.astype(str))

# マスタCSVファイルを保存
saved_df1.to_csv(output_dir1, index=False, encoding='cp932')
saved_df2.to_csv(output_dir2, index=False, encoding='cp932')

# マスタファイルを年度別にファイルを分けて保存（year列を削除）
for year, df in saved_df1.groupby('year'):
    df.drop(columns='year').to_csv(
        f"D:/Keiba/00_ImportData/170_Rating_Score/Rating_Score_Master_{year}.csv",
        index=False, encoding='cp932'
    )

for year, df in saved_df2.groupby('year'):
    df.drop(columns='year').to_csv(
        f"D:/Keiba/00_ImportData/50_Training_Comments/Training_Comments_Master_{year}.csv",
        index=False, encoding='cp932'
    )


C:\Users\rotte\AppData\Local\Temp\ipykernel_11304\731897491.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  master_df2 = pd.read_csv(output_dir2, encoding='cp932', dtype={'target_horseid': str}).copy()


In [28]:
# 出力先パス
output_dir1 = Path(r"D:\Keiba\00_ImportData\170_Rating_Score\Rating_Score_Master.csv")
output_dir2 = Path(r"D:\Keiba\00_ImportData\50_Training_Comments\Training_Comments_Master.csv")


rating_df.to_csv(output_dir1, index=False, encoding='cp932')
training_df.to_csv(output_dir2, index=False, encoding='cp932')

In [24]:

import race_data_pipeline2
importlib.reload(race_data_pipeline2)

training_df = race_data_pipeline2.scrape_kb_training(kb_raceid_list, target_raceid_list)

In [25]:
display(training_df)

,target_horseid,乗り役,日付,コース,馬場状態,8F,7F,6F,5F(4F),4F(3F),3F(2F),1F,回り位置,脚色,row_kind,training_comment
0,202509060604010101,松岡正,◇,函館Ｗ,重,,,,70.0,55.0,40.6,13.5,８,馬なり余力,time,松岡正 ◇ 函館Ｗ 重 70.0 55.0 40.6 13.5 ［８］ 馬なり余力
1,202509060604010101,松岡正,2025-09-03,札幌ダ,重,,,85.4,70.2,55.0,40.6,12.5,８,一杯に追う,oikiri,松岡正 2025-09-03 札幌ダ 重 85.4 70.2 55.0 40.6 12.5 ...
2,202509060604010102,助手,2025-08-24,函館Ｗ,重,,,,,59.8,44.1,14.4,８,馬なり余力,time,助手 2025-08-24 函館Ｗ 重 59.8 44.1 14.4 ［８］ 馬なり余力
3,202509060604010102,舟山瑠,2025-08-28,札幌ダ,不,,,,72.0,55.4,40.1,12.2,７,馬なり余力,time,舟山瑠 2025-08-28 札幌ダ 不 72.0 55.4 40.1 12.2 ［７］ 馬...
4,202509060604010102,舟山瑠,2025-09-03,札幌ダ,重,,,86.1,70.3,54.5,40.1,12.7,８,G前仕掛け,oikiri,舟山瑠 2025-09-03 札幌ダ 重 86.1 70.3 54.5 40.1 12.7 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11558,202509280904091215,助手,2025-09-24,美坂,良,,,,54.7,39.5,25.3,12.3,,一杯に追う,oikiri,助手 2025-09-24 美坂 良 54.7 39.5 25.3 12.3 一杯に追う
11559,202509280904091216,助手,■,栗坂,良,,,,55.3,40.2,25.6,12.1,,馬なり余力,time,助手 ■ 栗坂 良 55.3 40.2 25.6 12.1 馬なり余力
11560,202509280904091216,助手,2025-09-18,栗坂,良,,,,52.8,38.0,24.4,11.8,,一杯に追う,time,助手 2025-09-18 栗坂 良 52.8 38.0 24.4 11.8 一杯に追う
11561,202509280904091216,助手,2025-09-21,栗坂,重,,,,56.5,41.2,26.7,13.1,,馬なり余力,time,助手 2025-09-21 栗坂 重 56.5 41.2 26.7 13.1 馬なり余力


In [ ]:
import re
import pandas as pd
from bs4 import BeautifulSoup

OIKIRI_COLS = ['乗り役','日付','コース','馬場状態','8F','7F','6F','5F(4F)','4F(3F)','3F(2F)','1F','回り位置','脚色']

def _clean(t: str) -> str:
    return re.sub(r'\s+', ' ', (t or '')).strip()

def _page_ymd_from_text(txt: str):
    m = re.search(r'(\d{4})年\s*(\d{1,2})月\s*(\d{1,2})日', txt or '')
    return (int(m.group(1)), int(m.group(2)), int(m.group(3))) if m else None

def _norm_date(text: str, base_year: int | None) -> str:
    m = re.search(r'(\d{1,2})/(\d{1,2})', text or '')
    if not m or not base_year:
        return _clean(text)
    mm, dd = int(m.group(1)), int(m.group(2))
    return f'{base_year:04d}-{mm:02d}-{dd:02d}'

def _pick_alias(tr, tds, aliases: list[str], idx_fallback: int) -> str:
    for cls in aliases:
        el = tr.select_one(f'td.{cls}')
        if el:
            return _clean(el.get_text())
    if 0 <= idx_fallback < len(tds):
        return _clean(tds[idx_fallback].get_text())
    return ''

def _extract_laps_right_aligned(tds) -> list[str]:
    """セル中の 00.0 を拾い、7枠（8F..1F）に右詰めで詰める"""
    nums = []
    for td in tds:
        m = re.search(r'\d{1,2}\.\d', _clean(td.get_text()))
        if m: nums.append(m.group(0))
    out = [''] * 7
    for i, v in enumerate(reversed(nums[-7:])):
        out[-1 - i] = v
    return out

def _extract_horse_no_for_block(td_cyokyo) -> int | None:
    """同じブロックの見出しテーブル（class=cyokyo）から馬番を拾う"""
    # 直近の先祖/直前の table.cyokyo を優先的に参照
    tbl = td_cyokyo.find_parent('table', class_='cyokyo') or td_cyokyo.find_previous('table', class_='cyokyo')
    if not tbl:
        return None
    txt = tbl.get_text(' ', strip=True)
    m = re.search(r'枠\s*番\s*馬\s*番.*?\b(\d{1,2})\s+(\d{1,2})\s+', txt)
    if m:  # 2つ目が馬番
        return int(m.group(2))
    m2 = re.findall(r'\b(\d{1,2})\b', txt)
    return int(m2[1]) if len(m2) >= 2 else None

def parse_block_rows(html: str, target_raceid: str) -> list[dict]:
    """td.cyokyo > table.cyokyodata ＝ 1頭分ブロックから、time/oikiriの全行を抽出"""
    soup = BeautifulSoup(html, 'lxml')
    base_ymd = _page_ymd_from_text(soup.get_text(' ', strip=True))
    base_year = base_ymd[0] if base_ymd else None

    rows: list[dict] = []

    # 1頭ブロック単位で回す
    for td in soup.select('td.cyokyo'):
        horse_no = _extract_horse_no_for_block(td)
        target_horseid = f'{target_raceid}{horse_no:02d}' if horse_no is not None else ''

        tbl = td.select_one('table.cyokyodata')
        if not tbl:
            continue

        # time と oikiri をまとめて対象に（awase 等の説明行はスキップ）
        for tr in tbl.select('tr'):
            cls = tr.get('class') or []
            if not any(c in ('time', 'oikiri') for c in cls):
                continue

            tds = tr.select('td')

            rider  = _pick_alias(tr, tds, ['norite'], 1)
            nichi  = _pick_alias(tr, tds, ['nichi','tukihi'], 2)
            course = _pick_alias(tr, tds, ['course','corse'], 3)
            baba   = _pick_alias(tr, tds, ['baba'], 4)

            laps   = _extract_laps_right_aligned(tds)
            mawari = _pick_alias(tr, tds, ['mawariiti'], 12)
            asiiro = _pick_alias(tr, tds, ['akiiro','asiiro'], 13)

            row = {
                'target_horseid': target_horseid,
                '乗り役': rider,
                '日付': _norm_date(nichi, base_year),
                'コース': course,
                '馬場状態': baba,
                '8F': laps[0],
                '7F': laps[1],
                '6F': laps[2],
                '5F(4F)': laps[3],
                '4F(3F)': laps[4],
                '3F(2F)': laps[5],
                '1F': laps[6],
                '回り位置': mawari,
                '脚色': asiiro,
                # 行の種類が欲しければコレも残せる（あとで削除可）
                'row_kind': 'oikiri' if 'oikiri' in cls else 'time',
            }
            # 1本としてカウントできる行だけ（ラップが1つ以上ある）を採用
            if any(v for v in laps):
                rows.append(row)

    return rows

def build_training_df_from_html(html: str, target_raceid: str) -> pd.DataFrame:
    rows = parse_block_rows(html, target_raceid)
    df = pd.DataFrame(rows, columns=['target_horseid'] + OIKIRI_COLS + ['row_kind'])
    # training_comment（target_horseid/row_kind以外を連結）
    use_cols = [c for c in df.columns if c not in ('training_comment','target_horseid','row_kind')]
    df['training_comment'] = df[use_cols].apply(lambda s: ' '.join([str(v).strip() for v in s if str(v).strip()]), axis=1)
    return df


from pathlib import Path

html = Path(r"D:\Keiba\Python\調教 _ 2025年10月12日東京11R第１回　アイルランドトロフィー(ＧⅡ) _ 競馬ブック.html").read_text(encoding="utf-8", errors="ignore")
df = build_training_df_from_html(html, target_raceid="202510120411")  # 例
print(df.shape)
display(df)



(86, 16)


,target_horseid,乗り役,日付,コース,馬場状態,8F,7F,6F,5F(4F),4F(3F),3F(2F),1F,回り位置,脚色,row_kind,training_comment
0,20251012041101,24/9,ベスト,美坂,良,,,,53.8,39.8,25.5,12.5,,馬なり余力,time,24/9 ベスト 美坂 良 53.8 39.8 25.5 12.5 馬なり余力
1,20251012041101,助手,■,美Ｗ,良,,,85.1,68.2,52.9,37.7,11.6,［５］,馬なり余力,time,助手 ■ 美Ｗ 良 85.1 68.2 52.9 37.7 11.6 ［５］ 馬なり余力
2,20251012041101,田辺裕,◇,美Ｗ,良,,,82.7,66.7,51.4,37.2,11.2,［２］,馬なり余力,time,田辺裕 ◇ 美Ｗ 良 82.7 66.7 51.4 37.2 11.2 ［２］ 馬なり余力
3,20251012041101,杉原誠,2025-10-02,美Ｗ,稍,,,82.5,66.2,51.4,37.1,11.2,［２］,G前仕掛け,time,杉原誠 2025-10-02 美Ｗ 稍 82.5 66.2 51.4 37.1 11.2 ［...
4,20251012041101,助手,2025-10-05,美坂,稍,,,,58.4,42.2,27.2,13.0,,馬なり余力,time,助手 2025-10-05 美坂 稍 58.4 42.2 27.2 13.0 馬なり余力
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,20251012041116,23/12,ベスト,栗ＣＷ,良,,96.1,79.2,65.0,50.9,36.5,11.6,［４］,一杯に追う,time,23/12 ベスト 栗ＣＷ 良 96.1 79.2 65.0 50.9 36.5 11.6 ...
82,20251012041116,助手,■,栗ＣＷ,良,,,,72.3,55.7,38.6,11.2,［４］,馬なり余力,time,助手 ■ 栗ＣＷ 良 72.3 55.7 38.6 11.2 ［４］ 馬なり余力
83,20251012041116,助手,2025-10-01,栗ＣＷ,良,,98.4,81.4,65.9,51.2,36.7,11.3,［８］,強めに追う,time,助手 2025-10-01 栗ＣＷ 良 98.4 81.4 65.9 51.2 36.7 1...
84,20251012041116,助手,2025-10-05,栗ＣＷ,重,,,90.4,75.1,58.0,41.0,12.4,［９］,馬なり余力,time,助手 2025-10-05 栗ＣＷ 重 90.4 75.1 58.0 41.0 12.4 ［...


In [ ]:
#netkeibaからレースidを取得
#回数指定する場合
#race_id_list, target_race_id_list = GetRaceid.scrape_race_id_list(kaisai_date_list[:10])
#race_id_list, target_race_id_list = GetRaceid.scrape_race_id_list(kaisai_date_list[-10:])
nk_raceid_list,target_raceid_list = Get_nk_Raceid.scrape_nk_raceid_list(nk_kaisai_date_list)

In [ ]:
#netkeibaから調教データを取得する
Training_df = Get_nk_Traning.scrape_training_data(nk_raceid_list,target_raceid_list)

In [78]:
#調教データをcsvファイル出力する
Create_csv3 = Create_Impdata3.Create_Importdata3(Training_df)

CSVファイルが作成されました: D:\Keiba\Importdata\10_Traning_Comments\nk_training_comments_20241016.csv


# 前日データ取得２
## ・競馬ブックから指定した期間の各レースに出走する馬のレイティングを取得する
## ・取得したデータを元にTargetにインポートするためのCSVファイルを生成する

### スクレイピング対象データ
- 馬番
- レイティング

### ステップ
1. **netkeibaのカレンダーページから開催日を取得**
2. **開催日と固定のURL要素を組み合わせ、開催情報ページにアクセスし、レースIDを取得**
3. **開催日とレースIDを組み合わせて、Target取り込み用のレースIDを生成**
4. **競馬ブックの開催情報ページにアクセスし、レース詳細ページにアクセスするためのレースIDを取得**
5. **競馬ブックの出馬表ページにアクセス**
6. **出馬表ページから馬番を取得し、Target取り込み用の馬番付きレースIDを生成**
7. **出馬表ページから各馬のレイティングを取得し、以下の形でリストを生成**
   - `[馬番付きレースID, レイティング]`
8. **生成したリストをCSVファイルとして指定のディレクトリに出力**

In [ ]:
#netkeibaから開催日を取得する　yyyy-mm/yyyy-mmで取得期間を指定
nk_kaisai_date_list = Get_nk_Calendar.scrape_nk_kaisai_date(from_="2024-07",to_="2024-09")

In [ ]:
#netkeibaからレースidを取得
#回数指定する場合
#race_id_list, target_race_id_list = GetRaceid.scrape_race_id_list(kaisai_date_list[:10])
#race_id_list, target_race_id_list = GetRaceid.scrape_race_id_list(kaisai_date_list[-10:])
nk_raceid_list,target_raceid_list = Get_nk_Raceid.scrape_nk_raceid_list(nk_kaisai_date_list)

#競馬ブックからレースidを取得する
kb_raceid_list = Get_kb_Raceid.scrape_kb_raceid_list(nk_kaisai_date_list)

In [ ]:
#競馬ブックの出馬表から各馬のレイティングを取得する
rating_df = Get_kb_Rating.scrape_kb_racecard(kb_raceid_list,target_raceid_list)

In [ ]:
#レイティングをcsvファイル出力する
Create_Csv4 = Create_Impdata4.Create_Importdata4(rating_df)